<a href="https://colab.research.google.com/github/daniel-nuno/time_series_O2024_MAF3074N/blob/main/evaluacion_pronosticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

<br>
<br>
Clase: Series de tiempo
<br>
Actividad: Evaluación y Pronósticos
<br>
<br>

* * *

Docente: Daniel Nuño <br>
Fecha: 7 de octubre 2024 <br>

* * *

<br>
<br>

# Evaluación de pronósticos

Es importante evaluar la precisión de las previsiones utilizando previsiones auténticas. Por consiguiente, el tamaño de los residuos no es una indicación fiable de la magnitud de los errores de previsión reales. La precisión de las previsiones sólo puede determinarse teniendo en cuenta el rendimiento de un modelo con datos nuevos que no se utilizaron al ajustarlo.

Hay que tener en cuenta los siguientes puntos.

* Un modelo que se ajuste bien a los datos de entrenamiento no necesariamente pronosticará bien.
* Siempre se puede obtener un ajuste perfecto utilizando un modelo con suficientes parámetros.
* Ajustar demasiado un modelo a los datos es tan malo como no identificar un patrón sistemático en los datos.
* Algunas referencias describen el conjunto de prueba como el «conjunto de retención» porque estos datos se «retienen» de los datos utilizados para el ajuste.

Otras referencias llaman al conjunto de entrenamiento «datos dentro de la muestra» y al conjunto de prueba «datos fuera de la muestra». En este libro preferimos utilizar «datos de entrenamiento» y «datos de prueba».

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy as sp
import yfinance as yf

In [2]:
data = pd.read_excel("inflacion.xlsx")
data.index = pd.to_datetime(data['Fecha'])
del data['Fecha']


In [3]:
data.tail()

,values
Fecha,
2024-03-01,4.42
2024-04-01,4.65
2024-05-01,4.69
2024-06-01,4.98
2024-07-01,5.57


In [4]:
training_mask = data.index > '2023-01-01'

In [5]:
test = data[training_mask]
train = data[~training_mask]
horizon = len(test)

In [6]:
test.head()

,values
Fecha,
2023-02-01,7.62
2023-03-01,6.85
2023-04-01,6.25
2023-05-01,5.84
2023-06-01,5.06


In [7]:
pip install statsforecast

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas>=1.3.5 in c:\users\rogel\anaconda3\lib\site-packages (from statsforecast) (1.4.2)



In [8]:
from statsforecast.models import SeasonalNaive

In [11]:

train_values = train.values.flatten()  
model_sn = SeasonalNaive(season_length=12)
model_sn = model_sn.fit(y=train_values)
y_hat_dict = model_sn.predict(h=horizon)

y_hat_dict['mean']


array([7.28, 7.45, 7.68, 7.65, 7.99, 8.15, 8.7 , 8.7 , 8.41, 7.8 , 7.82,
       7.91, 7.28, 7.45, 7.68, 7.65, 7.99, 8.15], dtype=float32)

In [12]:
model_sn.predict_in_sample()

{'fitted': array([   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,   5.2 ,   4.81,   5.01,   4.87,
          5.09,   5.35,   5.46,   5.84,   5.11,   4.05,   4.6 ,   4.69,
          4.93,   5.38,   5.47,   5.87,   5.87,   5.71,   5.11,   5.58,
          5.67,   5.74,   5.34,   4.96,   4.4 ,   4.29,   4.46,   4.58,
          4.57,   4.87,   5.35,   5.08,   5.22,   5.19,   5.7 ,   5.56,
          6.62,   7.16,   7.52,   8.54,   9.47,   9.56,  11.95,  13.  ,
         15.16,  16.56,  17.23,  21.37,  23.92,  25.68,  25.54,  25.26,
         24.92,  25.13,  23.76,  23.09,  21.6 ,  22.44,  24.31,  20.6 ,
         17.92,  15.95,  15.79,  15.2 ,  15.83,  16.65,  15.91,  15.69,
         15.22,  13.56,  11.27,  11.31,  12.02,  13.49,  13.89,  13.72,
         13.01,  11.56,  11.62,  11.72,  14.69,  20.53,  25.1 ,  27.2 ,
         28.77,  29.19,  30.17,  31.22,  31.45,  32.53,  32.91,  34.35,
         32.22,  26.13,  22.  ,  20.66,  19.53,  18.63

In [13]:
residuals_train = train.values[12:] - model_sn.predict_in_sample()['fitted'][12:]

In [14]:
residuals_test = test.values - y_hat_dict['mean']

In [15]:
rmse_trian = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [16]:
tabla_modelos = pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'], data = [['Seasonal Naive', rmse_trian, rmse_test, 0]])

In [17]:
tabla_modelos

,Modelo,RMSE Train,RMSE Test,Parametros
0,Seasonal Naive,43.481444,2.923331,0


## RMSE

En machine learning, el RMSE (Root Mean Squared Error o Raíz del Error Cuadrático Medio) es una métrica comúnmente utilizada para evaluar el rendimiento de un modelo en problemas de regresión. Estas son algunas de sus principales ventajas:

* Penaliza los errores grandes: Como el RMSE eleva al cuadrado las diferencias
entre las predicciones y los valores reales antes de promediar, los errores más grandes tienen un impacto mayor en la métrica final. Esto es útil cuando deseas que el modelo sea particularmente sensible a los errores grandes.
* Fácil interpretación: El RMSE está en la misma escala que la variable de salida, lo que facilita su interpretación. Por ejemplo, si estás prediciendo precios de casas en miles de dólares, un RMSE de 5 significa que el error promedio es de 5,000 dólares.
* Sensibilidad a cambios pequeños: Debido a la elevación al cuadrado, el RMSE puede detectar cambios pequeños en las predicciones, lo que lo hace adecuado cuando se busca mejorar el modelo finamente.
* Comparación entre modelos: RMSE es una métrica estándar ampliamente utilizada, lo que facilita la comparación del rendimiento de diferentes modelos de regresión en un conjunto de datos específico.
* Diferenciable: En el contexto del entrenamiento de modelos basados en métodos de optimización como el gradiente descendente, el RMSE es una función diferenciable, lo que permite calcular su gradiente y optimizar los pesos del modelo de manera eficiente.

## R-cuadrada

La principal diferencia entre RMSE (Root Mean Squared Error) y R-cuadrada (Coeficiente de Determinación) radica en el tipo de información que ofrecen y en cómo se interpretan en el contexto de los modelos de regresión.

1. RMSE (Root Mean Squared Error)
Métrica de error: El RMSE mide directamente el error promedio de las predicciones de un modelo. Cuantifica la magnitud del error en las predicciones del modelo, representando la distancia media entre los valores observados y los valores predichos.
Interpretación: Se interpreta en las mismas unidades que la variable dependiente o de salida. Un RMSE más bajo indica un modelo más preciso, pero no proporciona un sentido claro de qué tan bien el modelo está explicando la variabilidad de los datos.
Escala: Depende de las unidades de la variable dependiente. Esto significa que puede ser difícil comparar el RMSE entre modelos aplicados a diferentes conjuntos de datos o variables de diferentes escalas.
2. R-cuadrada (Coeficiente de Determinación)
* Métrica de ajuste: mide qué proporción de la variabilidad total en los datos de salida está explicada por el modelo. No mide el error en sí, sino qué tan bien las predicciones del modelo capturan las tendencias generales en los datos.
* Interpretación: va de 0 a 1 (o puede ser negativo si el modelo es peor que una línea de base), donde:
*   =1 significa que el modelo predice perfectamente todos los puntos.

*  =0 significa que el modelo no predice mejor que la media de los datos (es decir, no explica nada de la variabilidad).
*   < 0 significa que el modelo es peor que simplemente usar la media como predicción.

* Escala relativa: Como es una proporción, es más fácil de interpretar y comparar entre diferentes modelos o conjuntos de datos, ya que no está influenciado por las unidades de la variable dependiente.

3. Comparación clave:
* RMSE se centra en la magnitud del error en las predicciones. Si el RMSE es pequeño, significa que el modelo predice bien en términos absolutos.
* R-cuadrada se enfoca en la proporción de la variabilidad de los datos que el modelo puede explicar. Un alto R-cuadrada significa que el modelo está capturando bien las tendencias generales de los datos, pero no garantiza que los errores individuales sean pequeños (ahí es donde entra el RMSE).

**En resumen:**

* RMSE te dice "cuánto se están equivocando las predicciones en promedio.
* R-cuadrada "te dice "qué porcentaje de la variabilidad en los datos está siendo explicado por el modelo."

Ambas métricas son útiles, pero proporcionan información diferente. Es común usar ambas para evaluar modelos de regresión.

$$
R_{2} = 1 - \frac{SS_{res}}{SS_{tot}}
$$

$ SS_{res} $: es la suma de los cuadrados de los residuos (pronósticos) (también llamado error residual)

$ SS_{tot} $: es la suma total de los cuadrados (media) (también llamado total de la variabilidad):

In [36]:
pd.concat([tabla_modelos, pd.DataFrame(columns=['Modelo', 'RMSE', 'Parametros'], data = [['Regresion', rmse, 0]])])

,Modelo,RMSE,Parametros
0,Seasonal Naive,467.073,0
0,Regresion,467.073,0
